In [1]:
# # Make sure the OpenAI library is installed
# %pip install openai

# # We'll need to install the Chroma client
# %pip install chromadb

# # Install wget to pull zip file
# %pip install wget

# # Install numpy for data manipulation
# %pip install numpy

# !pip install llama-index

### References:

1. Chromadb: https://docs.trychroma.com/guides

## Libraries

In [7]:
import openai
import pandas as pd
import os
import json
from tqdm import tqdm
from ast import literal_eval
import concurrent.futures

# Chroma's client library for Python
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

from openai import OpenAI

# Ignore unclosed SSL socket warnings - optional in case you get these errors
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## Configs

In [8]:
if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

EMBEDDING_MODEL = "text-embedding-3-small"
LLM_MODEL = "gpt-4o"
client = OpenAI()
chroma_client = chromadb.PersistentClient(path="./chromadb/")

embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'), model_name=EMBEDDING_MODEL)



OPENAI_API_KEY is ready


In [9]:
## Creating Collection
# chroma_client.delete_collection(name="cd-product_matcher")
# product_matcher_collection = chroma_client.create_collection(name='cd-product_matcher', 
#                                                              metadata={"hnsw:space": "cosine"},
#                                                              embedding_function=embedding_function)

In [10]:
## Loading Collection:
product_matcher_collection = chroma_client.get_collection(name="cd-product_matcher", embedding_function=embedding_function)

In [11]:
internal_product_csv_path = './data/Data_Internal.csv'

## Utils

In [12]:
def get_embedding(text, model=EMBEDDING_MODEL):
    if not text:
        raise ValueError("Input text cannot be empty.")
    try:
        text = text.replace("\n", " ")
        return client.embeddings.create(input = [text], model=model).data[0].embedding
    except Exception as e:
        print(f"An error occurred in get_embedding: {e}")
        raise

In [13]:
def add_embeddings_to_collection(df, collection, text_column='LONG_NAME', model=EMBEDDING_MODEL):
    """
    Adds embeddings of the specified text column from the dataframe to the Chroma collection.

    :param df: pandas DataFrame containing the data
    :param collection: Chroma collection to add the embeddings to
    :param text_column: Column name in the dataframe containing the text to embed
    :param model: Embedding model to use
    """
    exceptions = []

    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        text = row[text_column]
        if pd.notna(text):
            try:
                embedding = get_embedding(text, model=model)
                collection.add(
                    embeddings=[embedding],
                    metadatas=[{
                        "NAME": row.get("NAME", ""),
                        "OCS_NAME": row.get("OCS_NAME", ""),
                        "LONG_NAME": text
                    }],
                    ids=[f"id_{index}"]
                )
            except Exception as e:
                exceptions.append(f"Failed to add embedding for {text}: {e}")

    if exceptions:
        print("Exceptions occurred during processing:")
        for exception in exceptions:
            print(exception)

In [58]:
def query_collection(collection, query, max_results):
    """
    Queries the Chroma collection and returns the top results with metadata.

    :param collection: Chroma collection to query
    :param query: Query text
    :param max_results: Maximum number of results to return
    :return: DataFrame with the top results including NAME, OCS_NAME, and LONG_NAME
    """
    results = collection.query(query_texts=[query], n_results=max_results, include=['distances', 'metadatas'])
    
    # Flatten the results
    ids = results['ids'][0]
    scores = results['distances'][0]
    metadatas = results['metadatas'][0]
    
    # Create a DataFrame with the results
    result_df = pd.DataFrame({
        'id': ids,
        'score': scores,
        'NAME': [metadata.get('NAME', '') for metadata in metadatas],
        'OCS_NAME': [metadata.get('OCS_NAME', '') for metadata in metadatas],
        'LONG_NAME': [metadata.get('LONG_NAME', '') for metadata in metadatas]
    })
    
    return result_df

In [59]:
def match_products_and_get_response(external_product_information, potential_matching_internal_products):
    """
    Matches products using the OpenAI API and returns the response in JSON format.

    :param external_product_information: Information about the external product
    :param potential_matching_internal_products: List of potentially matching internal products
    :return: JSON response with match information
    """
    # Format the user prompt
    user_prompt = f"""
    Here is the external product information:
    {external_product_information}
    
    Here is the list of potentially matching:
    {potential_matching_internal_products}
    
    Please note that the response needs to be in json and in the required format.
    """
    
    # Call the OpenAI API
    try:
        messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ]

        # Call the chat completion endpoint with the constructed messages
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=messages,
            response_format={'type': 'json_object'}
        )

        # Extract and parse the response
        response_content = response.choices[0].message.content
        response_json = json.loads(response_content)

        return response_json

    except json.JSONDecodeError as e:
        print(f"Error decoding the JSON format: {e}")
        return {"is_matched": False, "match": "NULL"}
    except Exception as e:
        print(f"An error occurred while calling the OpenAI API: {e}")
        return {"is_matched": False, "match": "NULL"}

## R&D

In [60]:
internal_product_list = pd.read_csv(internal_product_csv_path)
internal_product_list.head()

,NAME,OCS_NAME,LONG_NAME
0,3 Mskt DkChocMnt 1.24oz,3 Mskt DkChocMnt 1.24oz,3 Musketeers Dark Chocolate Mint (1.24oz)
1,NaN,Costco Choc Mini 4.69lb,Costco Chocolate Mini (4.69lb)
2,Dove Dk Choc Bars 1.3oz,Dove Dk Choc Bars 1.3oz,Dove Dark Chocolate Bars (1.3oz)
3,Fishers ChocPnut 3.5oz,Fishers ChocPnut 3.5oz,Fisher's Chocolate Peanuts (3.5oz)
4,HariboGummiGoldBear2oz,HariboGummiGoldBear2oz,Haribo Gummi Gold-Bears (2oz)


In [61]:
# add_embeddings_to_collection(internal_product_list, product_matcher_collection)

In [62]:
# product_matcher_collection.peek()

In [63]:
product_matcher_collection.count()

16192

In [64]:
query = "MOUNTAIN DEW ZERO 20 OZ"
max_results = 10
top_results = query_collection(product_matcher_collection, query, max_results)
top_results

,id,score,NAME,OCS_NAME,LONG_NAME
0,id_1308,0.381033,MonsterZeroSugar16oz,MonsterZeroSugar16oz,Monster Energy Zero Sugar (16oz)
1,id_14379,0.384123,MonsterEngyZerUltra12oz,MonsterEngyZerUltra12oz,Monster Energy Zero Ultra (12oz)
2,id_2787,0.387438,MonsterAbsoluteZero24oz,,Monster Energy Absolutely Zero (24oz)
3,id_13885,0.388539,MonsterHydro0SgrWtr20oz,MonsterHydro0SgrWtr20oz,Monster Hydro Zero Sugar Energy Water (20oz)
4,id_4442,0.391420,MonsterZeroUltra 24oz,MonsterZeroUltra 24oz,Monster Energy Zero Ultra (24oz)
5,id_2682,0.398892,MonsterZeroUltra 16oz,MonsterZeroUltra 16oz,Monster Energy Zero Ultra (16oz)
6,id_11565,0.403540,,Monster Engy Zero Ult 10.5oz,Monster Energy Zero Ultra (10.5oz)
7,id_5644,0.403914,AmpEngyZeroWmelon16oz,AmpEngyZeroWmelon16oz,Amp Energy Zero Watermelon (16oz)
8,id_5931,0.405758,,CascadeIce0 Pch Mgo 17.2oz,Cascade Ice Zero Peach Mango (17.2oz)
9,id_11480,0.406790,PropelZeroPch 20oz,,Propel Zero Peach (20oz)


In [65]:
query = "YOO HOO CHOC. 11 OZ CANS"
max_results = 10
top_results = query_collection(product_matcher_collection, query, max_results)
top_results

,id,score,NAME,OCS_NAME,LONG_NAME
0,id_427,0.256742,Yoohoo Choc CN 11oz,Yoohoo Choc CN 11oz,Yoo-hoo Chocolate Drink Can (11oz)
1,id_343,0.307189,Yoohoo Choc Drink 12oz,Yoohoo Choc Drink 12oz,Yoo-hoo Chocolate Drink (12oz)
2,id_417,0.332007,Yoohoo Choc BT 15.5oz,Yoohoo Choc BT 15.5oz,Yoo-hoo Chocolate Drink Bottle (15.5oz)
3,id_5789,0.425638,,Cheerios Choc 11.25oz,Cheerios Chocolate (11.25oz)
4,id_5023,0.440203,,Cheerios Choc 22oz,Cheerios Chocolate (22oz)
5,id_1099,0.441297,HoodChocMlk 14oz,HoodChocMlk 14oz,Hood Chocolate Milk (14oz)
6,id_7441,0.460215,YoohooChocCandyBr4oz,,Yoo-hoo Milk Chocolate Flavored Candy Mini Bar...
7,id_1378,0.466774,Babcock Choc Mlk 16oz,,Babcock Chocolate Milk (16oz)
8,id_13158,0.468174,HoodWhlChocMlk 14oz,HoodWhlChocMlk 14oz,Hood Whole Chocolate Milk (14oz)
9,id_1813,0.470325,Monster Mlk Choc 14oz,,Monster Milk Chocolate (14oz)


In [66]:
SYSTEM_PROMPT = """
You are an intelligent assistant tasked with matching external products to internal products for a convenience store chain. Your goal is to find exact matches between the external product and the filtered list of internal products.

You will be provided with two inputs:

1. The external product information
2. A list of potentially matching internal products

Your task is to determine if there is an exact match between the external product and any of the internal products. An exact match means that the product manufacturer, name, and size must be identical.

To complete this task:

1. Carefully examine the external product information, noting the product name, manufacturer (if provided), and size.

2. Compare the external product to each internal product in the list. For each internal product, consider the NAME, OCS_NAME, and LONG_NAME fields to determine if there's an exact match.

3. Look for similarities in product name, manufacturer, and size. Pay close attention to abbreviations, variations in wording, and unit measurements.

4. If you find an exact match, where all relevant details align perfectly, mark it as a match.

5. If no exact match is found after comparing all internal products, conclude that there is no match.

Provide your response in JSON format with two keys:
- 'is_matched': Set to true if an exact match is found, false otherwise.
- 'match': If a match is found, provide the LONG_NAME of the matched internal product. If no match is found, set this to "NULL".

Your response should look like this:

<answer>
{
  "is_matched": <true/false>,
  "match": "<Matched Product LONG_NAME or NULL>"
}
</answer>

Here are some examples to guide you:

1. If the external product is "DIET LIPTON GREEN TEA W/ CITRUS 20 OZ" and an exact match is found:
{
  "is_matched": true,
  "match": "Lipton Diet Green Tea with Citrus (20oz)"
}

2. If the external product is "CH-CHERRY CHS CLAW DANISH 4.25 OZ" and an exact match is found:
{
  "is_matched": true,
  "match": "Cloverhill Cherry Cheese Bearclaw Danish (4.25oz)"
}

3. If the external product is "Hersheys Almond Milk Choco 1.6 oz" and no exact match is found:
{
  "is_matched": false,
  "match": "NULL"
}

Remember, the match must be exact. If you're unsure or if there's any discrepancy, err on the side of caution and report no match. Provide your final answer within the <answer> tags as shown above.
"""

In [67]:
def match_product_name(external_product_information, collection):
    """
    Main process to match external product information with internal products.

    :param external_product_information: Information about the external product
    :param collection: ChromaDB collection to query
    :param internal_product_list: DataFrame of internal products
    :return: JSON response with match information
    """
    # Query the collection to get the top 10 matches
    max_results = 10
    top_results = query_collection(collection, external_product_information, max_results)
    
    # Extract relevant fields for the top 10 matches
    potential_matching_internal_products = top_results[['NAME', 'OCS_NAME', 'LONG_NAME']].head(max_results).to_dict(orient='records')
    
    # Format the potential matching internal products as a string
    potential_matching_internal_products_str = "\n".join(
        [f"NAME: {item['NAME']}, OCS_NAME: {item['OCS_NAME']}, LONG_NAME: {item['LONG_NAME']}" for item in potential_matching_internal_products]
    )
    
    # Get the response from the match_products_and_get_response function
    response = match_products_and_get_response(external_product_information, potential_matching_internal_products_str)
    
    return response

In [68]:
# Example usage
external_product_info = "YOO HOO CHOC. 11 OZ CANS"
response = match_product_name(external_product_info, product_matcher_collection)
print(response)

{'is_matched': True, 'match': 'Yoo-hoo Chocolate Drink Can (11oz)'}


In [70]:
# Example usage
external_product_info = "MOUNTAIN DEW ZERO 20 OZ"
response = match_product_name(external_product_info, product_matcher_collection)
print(response)

{'is_matched': False, 'match': 'NULL'}
